# Práctica final Deep Learning: Poemas de Antonio Machado

## Objetivo

Desarrollo de una IA generativa capaz de generar poemas de Antonio Machado. Para ello se debe coger un archivo donde se puedan encontrar poemas de Antonio Machado con los que entrenar el modelo.

**Autores**
Ana Alonso Cañizares
Álvaro García Cid
Álvaro García Parra

## Desarrollo

### Implementación de librerías y funciones a utilizar

In [1]:
pip install fitz

In [2]:
pip install pymupdf

In [37]:
import fitz     # Para guardar en una variable los poemas encontrados en el documento
import re       # Tenemos en cuenta los saltos de línea como una palabra más
import string   # Para customizar la estandarización más adelante en la vectorización
import random

# Librerías necesarias para el desarrollo del modelo deep learning
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf

In [61]:
# Abrimos el fichero
pdf_doc = fitz.open("Poemas_Antonio_Machado.docx")

poemas = []

# En cada página del fichero se encuentra un poema
for page in pdf_doc:
  text = page.get_text()
  poemas.extend(text.strip().split("\n\n"))

# Observamos lo que se ha guardado
poemas

['Está en la sala familiar, sombría,\ny entre nosotros, el querido hermano\nque en el sueño infantil de un claro día\nvimos partir hacia un país lejano.\nHoy tiene ya las sienes plateadas,\nun gris mechón sobre la angosta frente;\ny la fría inquietud de sus miradas\nrevela un alma casi toda ausente.\nDeshójanse las copas otoñales\ndel parque mustio y viejo.\nLa tarde, tras los húmedos cristales,\nse pinta, y en el fondo del espejo.\nEl rostro del hermano se ilumina\nsuavemente. ¿Floridos desengaños\ndorados por la tarde que declina?\n¿Ansias de vida nueva en nuevos años?\n¿Lamentará la juventud perdida?\nLejos quedó —la pobre loba— muerta.\n¿La blanca juventud nunca vivida\nteme, que ha de cantar ante su puerta?\n¿Sonríe al sol de oro,\nde la tierra de un sueño no encontrada;',
 'y ve su nave hender el mar sonoro,\nde viento y luz la blanca vela henchida?\nÉl ha visto las hojas otoñales,\namarillas, rodar, las olorosas\nramas del eucalipto, los rosales\nque enseñan otra vez sus blancas

In [62]:
# Función que introduce un espacio antes y después de un "\n", o, lo que es lo mismo
# antes y después de cada salto de línea
def insert_spaces_before_after_newline(text):
    # Utiliza expresiones regulares para encontrar todas las ocurrencias de \n
    pattern = re.compile(r'(?<=\n)|(?=\n)')
    # Inserta un espacio antes y después de cada ocurrencia de \n
    modified_text = pattern.sub(' ', text)
    return modified_text

# Meter espacios alrededor de los saltos de línea
for i, poema in enumerate(poemas):
  poemas[i] = insert_spaces_before_after_newline(poemas[i])

poemas

['Está en la sala familiar, sombría, \n y entre nosotros, el querido hermano \n que en el sueño infantil de un claro día \n vimos partir hacia un país lejano. \n Hoy tiene ya las sienes plateadas, \n un gris mechón sobre la angosta frente; \n y la fría inquietud de sus miradas \n revela un alma casi toda ausente. \n Deshójanse las copas otoñales \n del parque mustio y viejo. \n La tarde, tras los húmedos cristales, \n se pinta, y en el fondo del espejo. \n El rostro del hermano se ilumina \n suavemente. ¿Floridos desengaños \n dorados por la tarde que declina? \n ¿Ansias de vida nueva en nuevos años? \n ¿Lamentará la juventud perdida? \n Lejos quedó —la pobre loba— muerta. \n ¿La blanca juventud nunca vivida \n teme, que ha de cantar ante su puerta? \n ¿Sonríe al sol de oro, \n de la tierra de un sueño no encontrada;',
 'y ve su nave hender el mar sonoro, \n de viento y luz la blanca vela henchida? \n Él ha visto las hojas otoñales, \n amarillas, rodar, las olorosas \n ramas del eucali

In [63]:
# Calcular el total de palabras en todos los poemas
total_palabras = sum(len(poema.split()) for poema in poemas)

# Calcular la media de palabras en total
media_palabras_total = total_palabras / len(poemas)

print("Media de palabras en total:", media_palabras_total)

Media de palabras en total: 109.71428571428571


In [64]:
max_length = round(media_palabras_total) # Tamaño de las secuencias
max_tokens = 1667  # Tamaño del vocabulario
batch_size = 4      # Tamaño del lote
embed_dim = 256     # Dimensión del embedding
num_heads = 2       # Número de cabezas del MultiHead
dense_dim = 32      # Nº de neuronas de la capa densa
EPOCHS = 30         # Nº de épocas

### 1º Forma

In [25]:
# Función que divide en dos el poema, con cuidado que no divida por mitad de una palabra
# La primera mitad corresponde al input_encoder
# La segunda mitad corresponde al input_decoder
def split_text(text):
    # Calcula la longitud total del texto
    text_length = len(text)

    # Encuentra el índice medio del texto
    middle_index = text_length // 2

    # Retrocede desde el índice medio hasta el inicio del texto y encuentra el espacio más cercano
    start_index = text.rfind(' ', 0, middle_index)

    # Retrocede desde el índice medio hasta el final del texto y encuentra el espacio más cercano
    end_index = text.find(' ', middle_index)

    # Si no se encontraron espacios antes o después del índice medio, ajusta los índices para dividir entre palabras
    if start_index == -1:
        start_index = 0
    if end_index == -1:
        end_index = text_length

    # Divide el texto en dos partes
    first_half = text[:end_index]
    second_half = text[end_index:]

    return first_half, second_half

In [26]:
# Agrupamos por pares las dos mitades
# A la segunda mitad le añadimos [start] al principio
# y [end] al final
poema_pairs = []
for poema in poemas:
  poema_mitad1, poema_mitad2 = split_text(poema)
  poema_pairs.append((poema_mitad1, "[start]" + poema_mitad2 + " [end]"))

In [27]:
# Separar el conjunto en entrenamiento, validación y test
# 15% validación
num_val_samples = int(0.15 * len(poema_pairs))
# 70% entrenamiento
num_train_samples = len(poema_pairs) - 2 * num_val_samples
train_pairs = poema_pairs[:num_train_samples]
val_pairs = poema_pairs[num_train_samples:num_train_samples + num_val_samples]
# 15% test o pruebas
test_pairs = poema_pairs[num_train_samples + num_val_samples:]

In [28]:
# Prepara una función de estandarización de cadenas
# personalizada para la capa TextVectorization en
# español: conserva [ y ] pero elimina ¿ y ¡ (así como
# todos los demás caracteres de cadenas.puntuación)
# Poemas
strip_chars = string.punctuation + "¿" + "¡"
strip_chars = strip_chars.replace("\\", "")

# Poemas con [start] y [end]
strip_chars2 = string.punctuation + "¿" + "¡"
strip_chars2 = strip_chars2.replace("[", "")
strip_chars2 = strip_chars2.replace("]", "")
strip_chars2 = strip_chars2.replace("\\", "")

# Poemas
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
    standardize=custom_standardization,
)

# Poemas con [start] y [end]
def custom_standardization_2(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars2)}]", "")

text_vectorization_start_end = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    # Generamos poemas que tengan un token
    # adicional, ya que necesitaremos compensar la oración
    # en un paso durante el entrenamiento
    output_sequence_length=max_length + 1,
    standardize=custom_standardization_2,
)

train_poemas = [pair[0] for pair in train_pairs]
train_poemas_start_end = [pair[1] for pair in train_pairs]

# Aprende el vocabulario de los poemas
text_vectorization.adapt(train_poemas)
text_vectorization_start_end.adapt(train_poemas_start_end)

In [29]:
def format_dataset(p, p_s_e):
    p = text_vectorization(p)
    p_s_e = text_vectorization_start_end(p_s_e)
    return ({
        "poema": p,
        # El poema de entrada
        # no incluye el último token para
        # mantener las entradas y los
        # objetivos en la misma longitud.
        "poema_start_end": p_s_e[:, :-1],
    # El poema objetivo está un
    # paso por delante. Ambos siguen siendo
    # de la misma longitud
    }, p_s_e[:, 1:])

def make_dataset(pairs):
    poema_texts, poema_start_end_texts = zip(*pairs)
    poema_texts = list(poema_texts)
    poema_start_end_texts = list(poema_start_end_texts)
    dataset = tf.data.Dataset.from_tensor_slices((poema_texts, poema_start_end_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    # Utilizamos el almacenamiento en caché en memoria
    # para acelerar el preprocesamiento
    return dataset.shuffle(2048).prefetch(16).cache()

# Construimos los dataset en un tensor de 2 dimensiones (input, target)
# input a su vez se divide en input_encoder e input_decoder
# target es el poema al que el Transformer debería llegar
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [30]:
# Observamos que esté todo en orden
for inputs, targets in train_ds.take(1):
  print(f"inputs['poema'].shape: {inputs['poema'].shape}")
  print(f"inputs['poema_start_end'].shape: {inputs['poema_start_end'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['poema'].shape: (4, 110)
inputs['poema_start_end'].shape: (4, 110)
targets.shape: (4, 110)


In [31]:
# Definimos el Encoder
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding
        # será 2D, pero la capa de atención espera
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [32]:
# Definimos el Decoder
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Este atributo asegura que la capa propagará
        # su máscara de entrada a sus salidas; el
        # enmascaramiento en Keras es explícitamente
        # opt-in. Si pasa una máscara a una capa que
        # no implementa compute_mask() y que no expone
        # este atributo support_masking, es un error.
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [33]:
# Definimos el positional embedding
# A la capa embedding se le ha añadido el parámetro mask_zero = True
# Para que no tenga en cuenta los 0 generados por el zero padding
class PositionalEmbedding(layers.Layer):
    # Una desventaja de las incrustaciones de posición es que
    # la longitud de la secuencia debe conocerse de antemano
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Prepara una capa de embedding para los índices de token.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim, mask_zero = True)
        self.position_embeddings = layers.Embedding(
            # Y otro para las posiciones te tokens
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # Agrega ambos vectores embeddings juntos
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Al igual que la capa de embedding,
        # esta capa debería poder generar una
        # máscara para que podamos ignorar los
        # ceros de relleno en las entradas.
        # El framework llamará automáticamente
        # al método compute_mask y la máscara
        # se propagará a la siguiente capa.
        return tf.math.not_equal(inputs, 0)

    # Implementamos la serialización para que
    # podamos guardar el modelo.
    def get_config(self):
        config = super().get_config()
        # config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [34]:
# Transformer encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name = "poema")
x = PositionalEmbedding(max_length, max_tokens, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Transformer decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name = "poema_start_end")
x = PositionalEmbedding(max_length, max_tokens, embed_dim)(decoder_inputs)
# Codificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)

# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(max_tokens, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
transformer.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 poema (InputLayer)          [(None, None)]               0         []                            
                                                                                                  
 poema_start_end (InputLaye  [(None, None)]               0         []                            
 r)                                                                                               
                                                                                                  
 positional_embedding_2 (Po  (None, None, 256)            464640    ['poema[0][0]']               
 sitionalEmbedding)                                                                               
                                                                                            

In [35]:
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/30
9/9 [==============================] - 8s 391ms/step - loss: 6.8699 - accuracy: 0.0356 - val_loss: 6.4203 - val_accuracy: 0.0455
Epoch 2/30
9/9 [==============================] - 2s 243ms/step - loss: 6.1502 - accuracy: 0.0642 - val_loss: 6.3880 - val_accuracy: 0.0668
Epoch 3/30
9/9 [==============================] - 2s 228ms/step - loss: 5.7382 - accuracy: 0.1033 - val_loss: 6.2978 - val_accuracy: 0.0775
Epoch 4/30
9/9 [==============================] - 2s 205ms/step - loss: 5.2065 - accuracy: 0.1499 - val_loss: 6.2287 - val_accuracy: 0.0802
Epoch 5/30
9/9 [==============================] - 3s 319ms/step - loss: 4.7243 - accuracy: 0.1890 - val_loss: 6.2460 - val_accuracy: 0.0722
Epoch 6/30
9/9 [==============================] - 3s 306ms/step - loss: 4.2358 - accuracy: 0.2466 - val_loss: 6.2299 - val_accuracy: 0.0829
Epoch 7/30
9/9 [==============================] - 3s 347ms/step - loss: 3.8159 - accuracy: 0.3043 - val_loss: 6.2427 - val_accuracy: 0.0775
Epoch 8/30
9/9 [====

In [38]:
# Vamos a probar que tal genera poemas nuestro modelo
poema_vocab = text_vectorization_start_end.get_vocabulary()
spa_index_lookup = dict(zip(range(len(poema_vocab)), poema_vocab))
max_decoded_sentence_length = max_length

def decode_sequence(input_sentence):
    tokenized_input_sentence = text_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = text_vectorization_start_end(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        # Muestra el siguiente token
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertimos la siguiente predicción del token en una
        # cadena y la agregamos a la oración generada
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Condición de salida
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Hoy, en mitad de la vida, 
 me he parado a meditar... 
 ¡Juventud nunca vivida 
 quién te volviera a soñar! 
 Eran ayer mis dolores 
 como gusanos de seda 
 que iban labrando capullos; 
 hoy son mariposas negras. 
 ¡De cuántas flores amargas 
 ha sacado blanca cera! 
 ¡Oh tiempo en que
[start] no sé qué triste es tu suerte medrosas tiritan tus hojas de la primavera tú has dicho el secreto que en mi alma cuando bosteza el viento agita el llanto es una voz o un eco para escuchar tu queja de la clase en la una [end]
-
que llegaron al alma, al hondo cielo? 
 ¿Y ha de morir contigo el mundo tuyo, 
 la vieja vida en orden tuyo y nuevo? 
 ¿Los yunques y crisoles de tu alma 
 trabajan para el polvo y para el viento? 
 Desnuda está la tierra, 
 y el alma aúlla al horizonte pálido 
 como loba famélica. ¿Qué buscas, 
 poeta, en el ocaso? 
 Amargo caminar, porque el camino
[start] maduro o florido de frondas y aromas y espuma ríen los zumos de la plaza muerta— viene a encender las rosas rojas de

### 2ª Forma

Tal y como se observa en las generaciones anteriores, se repiten las palabras en muchos de ello. Para evitar esto, se debe de otorgar algo de creatividad al modelo, para ello hay que meter cierta aleatoriedad a la hora de asignar cual va a ser la siguiente palabra.

En este modelo de IA generativa, solo tenemos un dataset, los poemas, no es necesario dividir estos en train, val y test.

Por ello, el primer paso es generar el dataset a partir del array de los poemas.

In [65]:
def make_dataset(poemas):
    poemas = list(poemas)
    dataset = tf.data.Dataset.from_tensor_slices(poemas)
    dataset = dataset.batch(batch_size)
    # Utilizamos el almacenamiento en caché en memoria
    # para acelerar el preprocesamiento
    return dataset.shuffle(2048).prefetch(16).cache()

# Construimos los dataset en un tensor de 2 dimensiones (input, target)
# input a su vez se divide en input_encoder e input_decoder
# target es el poema al que el Transformer debería llegar
poemas_ds = make_dataset(poemas)

In [66]:
strip_chars = string.punctuation + "¿" + "¡"
strip_chars = strip_chars.replace("\\", "")

# Poemas
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# Creamos la vectorización
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    # Queremos devolver secuencias de índice
    # de palabras con valores enteros
    output_mode="int",
    output_sequence_length=max_length,
    standardize=custom_standardization,
)

text_vectorization.adapt(poemas_ds)

In [67]:
def prepare_lm_dataset(poemas):
    # Convertimos un lote de textos (cadenas)
    # en un lote de secuencias enteras.
    vectorized_sequences = text_vectorization(poemas)
    # Creamos entradas cortando la última palabra de las secuencias
    x = vectorized_sequences[:, :-1]
    # Creamos objetivos compensando las secuencias por 1.
    y = vectorized_sequences[:, 1:]
    return x, y

poemas_ds = poemas_ds.map(prepare_lm_dataset, num_parallel_calls=4)

In [68]:
# Definimos el Decoder
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Este atributo asegura que la capa propagará
        # su máscara de entrada a sus salidas; el
        # enmascaramiento en Keras es explícitamente
        # opt-in. Si pasa una máscara a una capa que
        # no implementa compute_mask() y que no expone
        # este atributo support_masking, es un error.
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [69]:
# Definimos el positional embedding
# A la capa embedding se le ha añadido el parámetro mask_zero = True
# Para que no tenga en cuenta los 0 generados por el zero padding
class PositionalEmbedding(layers.Layer):
    # Una desventaja de las incrustaciones de posición es que
    # la longitud de la secuencia debe conocerse de antemano
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Prepara una capa de embedding para los índices de token.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim, mask_zero = True)
        self.position_embeddings = layers.Embedding(
            # Y otro para las posiciones te tokens
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # Agrega ambos vectores embeddings juntos
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Al igual que la capa de embedding,
        # esta capa debería poder generar una
        # máscara para que podamos ignorar los
        # ceros de relleno en las entradas.
        # El framework llamará automáticamente
        # al método compute_mask y la máscara
        # se propagará a la siguiente capa.
        return tf.math.not_equal(inputs, 0)

    # Implementamos la serialización para que
    # podamos guardar el modelo.
    def get_config(self):
        # config = super().get_config()
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [70]:
# Transformer decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(max_length, max_tokens, embed_dim)(decoder_inputs)
# Codificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, x)

# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(max_tokens, activation="softmax")(x)
model = keras.Model(decoder_inputs, decoder_outputs)

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy")
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding_6 (Po  (None, None, 256)            454912    ['input_5[0][0]']             
 sitionalEmbedding)                                                                               
                                                                                                  
 transformer_decoder_5 (Tra  (None, None, 256)            1070368   ['positional_embedding_6[0][0]
 nsformerDecoder)                                                   ',                            
                                                                     'positional_embedding_6

In [73]:
# Dict (diccionario) que asigna o mapea índices de palabras
# a cadenas, para usarlos para decodificar texto
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

# Implementa el muestreo de variable-temperatura
# a partir de una distribución de probabilidad
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 # Indicación o prompt que usamos para
                 # iniciar la generación de texto
                 prompt,
                 # Cuantas palabras generar
                 generate_length,
                 model_input_length,
                 # Rango de temperaturas
                 # a usar para el muestreo
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = text_vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # Al generar texto, comenzamos
            # desde nuestro prompt o aviso.
            sentence = self.prompt
            for i in range(self.generate_length):
                # Alimentamos nuestro modelo con la secuencia actual
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # Recuperamos las predicciones del último timestep y las
                # utilizamos para muestrear o probar una nueva palabra.
                next_token = sample_next(
                    # O [0, i, :]
                    predictions[0, self.prompt_length - 1 + i, :]
                )
                sampled_token = tokens_index[next_token]
                # Agregamos la nueva palabra a
                # la secuencia actual y repetimos
                sentence += " " + sampled_token
            print(sentence)

prompt = "En el mar"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=max_length,
    # Usaremos un rango diverso de
    # temperaturas para muestrear texto,
    # para demostrar el efecto de la
    # temperatura en la generación de texto.
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [74]:
model.fit(poemas_ds, epochs=20, callbacks=[text_gen_callback])

Epoch 1/20
13/13 [==============================] - ETA: 0s - loss: 5.3070== Generating with temperature 0.2
En el mar aire las —juguetes ya llanto quisiera a la emerge penetró duerme ancho gloria agua pierde nubes almas el viejo humo detén mal nuevo que de la sala laguna alma lo desde puros lágrimas de bien serpea que en solitaria con dista canción galerías la hastío la —contigo aceros humilde y
== Generating with temperature 0.5
En el mar tú de agudas mañana un empecé clara el maduro aura los pena todavía verán nunca nube y mundo el de al alcanzar de hondas la cipresal hacen las tictac cuando el lienzos mar rincón llamar blancas aire obscurece o casi corría primavera caminos letra  vidrio en sol de en
== Generating with temperature 0.7
En el mar míos los naranjo quieren aleja lo reflejo ella  ha dentro algún de luna los muda y dos querido humareda seco recuerdo los música que que el rubia amplia corazón salir viento carne el casi el ha los he se soberbios  conozco y hilo ser y irá lo

## Conclusiones

Los modelos generan poemas sin mucho sentido y sin conexión entre las palabras, esto se debe a las siguientes dos razones principalemente:
- Escasez de datos
- Word embedding y vocabulario insuficiente, este viene ligado con el primer punto, ya que al disponer de pocos datos no podemos generar un extenso vocabulario y/o word embedding donde ver la relación semántica de las palabras.

Si se disponiera de una BBDD más grande, seguramente la generación de poemas tendría más cohesión. Otra solución o punto a mejorar, es añadir un vocabulario y/o word embbeding, similar al de Antonio Machado, ya preentrenado.

Además, se ha tratado de preservar los saltos de línea para que el modelo aprendiese también a realizar dichos saltos, sin embargo, pasa desapercibido.